# Parsing votes
> Downloading & parsing votes Aafter downloading xlsx files behind the links on `https://www.bundestag.de/parlament/plenum/abstimmung/liste`.

## How to use

You can run this notebook to re-do some data processing. But know you can also skip this by running

    uv run bundestag download huggingface

instead.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import bundestag.data as b_data
from bundestag.data.download.bundestag_sheets import (
    RE_HTM,
    RE_FNAME,
    collect_sheet_uris,
    download_sheet,
    download_multiple_sheets,
)
from bundestag.data.transform.bundestag_sheets import (
    get_file2poll_maps,
    get_sheet_df,
    get_squished_dataframe,
    set_sheet_dtypes,
    get_multiple_sheets_df,
)
import bundestag.data.utils as data_utils
import bundestag.logging as logging
import bundestag.paths as paths

## Setup

In [ ]:
logger = logging.logger
logger.setLevel("DEBUG")

_paths = paths.get_paths("../data")
_paths

In [ ]:
dry = True

In [ ]:
_paths.make_raw_paths(dry=False)

## Collecting URIs for `.xlsx`/`.xls` documents from `.htm` files

`.xlsx` / `.xls` will be referred as "sheet" files.

In [ ]:
html_path = _paths.raw_bundestag_html
html_path

In [ ]:
html_file_paths = data_utils.get_file_paths(html_path, pattern=RE_HTM)
html_file_paths[:3]

In [ ]:
sheet_uris = collect_sheet_uris(html_file_paths)
list(sheet_uris.items())[:3], list(sheet_uris.items())[-3:]

## Downloading sheet files

In [ ]:
sheet_path = _paths.raw_bundestag_sheets

In [ ]:
uri = sheet_uris[
    "10.09.2020: Abstrakte Normenkontrolle - Düngeverordnung (Beschlussempfehlung)"
]
download_sheet(uri, sheet_dir=sheet_path, dry=dry)

In [ ]:
download_multiple_sheets(sheet_uris, sheet_dir=sheet_path, nmax=3, dry=dry)

In [ ]:
file_title_maps = get_file2poll_maps(sheet_uris, sheet_path)
file_title_maps

## Loading sheets into DataFrames

Collecting the `xlsx` and `xls` file names

In [ ]:
sheet_files = data_utils.get_file_paths(sheet_path, pattern=RE_FNAME)
sheet_files

Reading files into dataframes

In [ ]:
sheet_file = sheet_files[0]
sheet_file

In [ ]:
df = get_sheet_df(sheet_file, file_title_maps=file_title_maps)
df.head().T

Squishing vote columns

In [ ]:
df_squished = get_squished_dataframe(df)
df_squished.head(3).T

Setting some dtypes

In [ ]:
df_squished = set_sheet_dtypes(df_squished)

In [ ]:
df_squished.head().T

Loading multiple sheets into dataframes

In [ ]:
df = get_multiple_sheets_df(sheet_files, file_title_maps=file_title_maps)
df.head().T

Writing to disk

In [ ]:
_paths.preprocessed_bundestag

In [ ]:
df_path = _paths.preprocessed_bundestag / "bundestag.de_votes.parquet"
logger.info(f"Writing to {df_path}")

In [ ]:
_paths.make_preprocessed_paths(dry=False)

In [ ]:
df.to_parquet(df_path)